In [15]:
from kendama_env import KendamaEnv
import time
env = KendamaEnv()


start_time = time.time()
while 1:
    current_time = time.time()
    elapsed_time = current_time - start_time

    if elapsed_time > 1.0/240.0:
        start_time = time.time()
        action = [[0,0,0],[0,0,0]]
        ob, reward, done, _ = env.step(action)
    

error: Not connected to physics server.

In [ ]:
import pybullet as p
import pybullet_data as pd
import os

p.connect(p.DIRECT)
name_in = "C:/Users/arnau/Documents/Cours 3A/INF581/Projet/git/Kendama RL/URDF/kendama_ball/meshes/kendama_ball.obj"
name_out = "C:/Users/arnau/Documents/Cours 3A/INF581/Projet/git/Kendama RL/URDF/kendama_ball/meshes/base_link_decomposed.obj"
name_log = "log.txt"
p.vhacd(name_in, name_out, name_log,resolution=64000000,convexhullDownsampling=1,planeDownsampling=1,depth=32, concavity=0)